In [109]:
import sqlite3
from frozendict import frozendict
db = sqlite3.connect('rxnorm.sqlite')
def dict_factory(cursor, row):
    d = {}
    for idx, col in enumerate(cursor.description):
        d[col[0]] = row[idx]
    return d

db.row_factory = dict_factory
c = db.cursor()
from collections import defaultdict 


"""
with pill_leaves as (with recursive kids(name) as (
 select distinct rxcui1 from rxnrel where rela='doseformgroup_of' and rxcui2="1151133"
    union
 select rxcui2 from kids, rxnrel where rxnrel.rela="isa" and rxnrel.rxcui1=kids.name
) select distinct name from kids)
select count(*), tty from rxnconso where rxcui in (select name from pill_leaves) group by tty order by count(*);


with pill_leaves as (with recursive kids(name) as (
 select distinct rxcui1 from rxnrel where rela='doseformgroup_of' and rxcui2="1151133"
    union
 select rxcui2 from kids, rxnrel where rxnrel.rela="isa" and rxnrel.rxcui1=kids.name
) select distinct name from kids)
select * from rxnconso where rxcui in (select name from pill_leaves) limit 200;

"""


'\nwith pill_leaves as (with recursive kids(name) as (\n select distinct rxcui1 from rxnrel where rela=\'doseformgroup_of\' and rxcui2="1151133"\n    union\n select rxcui2 from kids, rxnrel where rxnrel.rela="isa" and rxnrel.rxcui1=kids.name\n) select distinct name from kids)\nselect count(*), tty from rxnconso where rxcui in (select name from pill_leaves) group by tty order by count(*);\n\n\nwith pill_leaves as (with recursive kids(name) as (\n select distinct rxcui1 from rxnrel where rela=\'doseformgroup_of\' and rxcui2="1151133"\n    union\n select rxcui2 from kids, rxnrel where rxnrel.rela="isa" and rxnrel.rxcui1=kids.name\n) select distinct name from kids)\nselect * from rxnconso where rxcui in (select name from pill_leaves) limit 200;\n\n'

In [110]:
c.execute("""
select * from rxnconso where 
cvf='4096' and 
rxcui in
  (select distinct rxcui1 from rxnrel where
      rela='doseformgroup_of' and 
      rxcui2="1151133")
and tty in ('SCDG');
""")
generic_drugs = c.fetchall()

c.execute("""
select * from rxnconso where 
cvf='4096' and 
rxcui in
  (select distinct rxcui1 from rxnrel where
      rela='doseformgroup_of' and 
      rxcui2="1151133")
and tty in ('SBDG');
""")
brand_drugs = c.fetchall()

print len(generic_drugs) + len(brand_drugs)

c.execute("""
select * from rxnconso where 
cvf='4096' and tty='PSN';
""")
preferred_terms = {x['RXCUI']: x for x in c.fetchall()}

c.execute("""
select * from rxnconso where 
cvf='4096'
and tty in ('SBDG', 'SCDG', 'SBD', 'SCD', 'SCDC', 'SBDC');
""")
all_drugs = {x['RXCUI']: x for x in c.fetchall()}
for (k,v) in preferred_terms.iteritems():
    if k not in all_drugs: continue
    all_drugs[k]['PREFERRED'] = v['STR']

pill_to_component_sets = defaultdict(set)
parts_to_prescribables = defaultdict(set)

def to_type(sources, target):
    subs = [x for x in sources]
    subs.extend([target])
    c = db.cursor()
    q = "select str, rxcui from rxnconso where rxcui in (select rxcui1 from rxnrel where rxcui2 in (%s) ) and sab='RXNORM' and tty= ? and cvf='4096' "%", ".join("?" * (len(subs) - 1))
    #print q
    #print subs
    c.execute(q, subs)
    return [v['RXCUI'] for v in c.fetchall()]

4068


In [114]:
all_drugs['863856']

{'CODE': u'863856',
 'CVF': u'4096',
 'ISPREF': u'',
 'LAT': u'ENG',
 'LUI': u'',
 'PREFERRED': u'morphine sulfate 80 MG / naltrexone HCl 3.2 MG Extended Release Oral Capsule',
 'RXAUI': u'6346936',
 'RXCUI': u'863856',
 'SAB': u'RXNORM',
 'SAUI': u'6346936',
 'SCUI': u'863856',
 'SDUI': u'',
 'SRL': u'',
 'STR': u'Morphine Sulfate 80 MG / Naltrexone hydrochloride 3.2 MG Extended Release Oral Capsule',
 'STT': u'',
 'SUI': u'',
 'SUPPRESS': u'N',
 'TS': u'',
 'TTY': u'SCD'}

In [111]:
# scdg -[inverse_isa]-> scd -[consists_of]-> scdc

for d in [g['RXCUI'] for g in generic_drugs]:
    scds = to_type([d], "SCD")
    for scd in scds:
        parts = to_type([scd], "SCDC")
        partskey = frozenset(parts)
        parts_to_prescribables[partskey].add(scd)
        pill_to_component_sets[d].add(partskey)

for d in [g['RXCUI'] for g in brand_drugs]:
    sbds = to_type([d], "SBD")
    for sbd in sbds:
        parts = to_type([sbd], "SBDC")
        partskey = frozenset(parts)
        parts_to_prescribables[partskey].add(sbd)
        pill_to_component_sets[d].add(partskey)

In [129]:
all_drugs['330863']

{'CODE': u'330863',
 'CVF': u'4096',
 'ISPREF': u'',
 'LAT': u'ENG',
 'LUI': u'',
 'RXAUI': u'1507629',
 'RXCUI': u'330863',
 'SAB': u'RXNORM',
 'SAUI': u'1507629',
 'SCUI': u'330863',
 'SDUI': u'',
 'SRL': u'',
 'STR': u'Lactase 9000 UNT',
 'STT': u'',
 'SUI': u'',
 'SUPPRESS': u'N',
 'TS': u'',
 'TTY': u'SCDC'}

In [94]:
print len(filter(lambda x: x==0, [len(pill_to_component_sets[v['RXCUI']]) for v in brand_drugs]))

for v in brand_drugs:
    if len(pill_to_component_sets[v['RXCUI']]) == 0:
        print v['STR']


0


In [125]:
json_db = {
    'cuiToName': {},
    'pillToComponentSets': {},
    'componentSetsToPrescribables': {}
}

def name_of(cui):
    r = all_drugs[cui]
    #if "PREFERRED" in r:
    #    return r["PREFERRED"]
    return r["STR"]

def sort_cuis(s):
    return sorted(s, lambda a, b:  -1 if all_drugs[a]['STR'] < all_drugs[b]['STR'] else 1)

def quote_component_set(s):
    return ",".join(sort_cuis(s))

for (pill, component_sets) in pill_to_component_sets.iteritems():
    json_db['cuiToName'][pill] = name_of(pill)
    json_db['pillToComponentSets'][pill] = [sort_cuis(cs) for cs in component_sets]
    for components in component_sets:
        for c in components:
            json_db['cuiToName'][c] = name_of(c)

for (component_set, prescribables) in parts_to_prescribables.iteritems():
    for p in prescribables:
        json_db['cuiToName'][p] = name_of(p)
    json_db['componentSetsToPrescribables'][quote_component_set(component_set)] = list(prescribables)


In [126]:
import json
extract = json.dumps(json_db)
len(extract)

1709732

In [ ]:
extract = [{'str': d['STR'][:-5], 'cui': d['RXCUI']} for d in  drugs]
print len(extract), extract[100]

In [127]:
import json
dump = json.dumps(json_db, indent=2)
with open('/tmp/rxnorm.js', 'w') as out:
    out.write("var db = %s;\nexport default db;"%dump)
